In [348]:
import imaplib
import email
from datetime import datetime, timedelta
import pytz
import time as t


import yaml  #To load saved login credentials from a yaml file

In [349]:
with open("credentials.yml") as f:
    content = f.read()

In [350]:
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)

In [351]:

#Load the user name and passwd from yaml file
gmail, gmail_password = my_credentials["gmail_user"], my_credentials["gmail_password"]

In [352]:
#URL for IMAP connection
imap_url = 'imap.gmail.com'
# Connection with GMAIL using SSL
my_mail = imaplib.IMAP4_SSL(imap_url)


In [353]:
try:
    my_mail.login(gmail, gmail_password)
    print ("LOGIN SUCCESSFUL!!! ")
    # Select the Inbox to fetch messages
    my_mail.select('Inbox')
except imaplib.IMAP4.error:
    print ("LOGIN FAILED!!! ")

LOGIN SUCCESSFUL!!! 


In [354]:

#Define Key and Value for email search
#For other keys (criteria): https://gist.github.com/martinrusev/6121028#file-imap-search
key = 'FROM'
value = 'jrajpal2@illinois.edu'
_, data = my_mail.search(None, key, value)  #Search for emails with specific key and value

mail_id_list = data[0].split()  #IDs of all emails that we want to fetch 





In [355]:
msgs = [] # empty list to capture all messages
#Iterate through messages and extract data into the msgs list
for num in mail_id_list:
    typ, data = my_mail.fetch(num, '(RFC822)') #RFC822 returns whole message (BODY fetches just body)
    msgs.append(data)

In [356]:
#Now we have all messages, but with a lot of details
#Let us extract the right text and print on the screen

#In a multipart e-mail, email.message.Message.get_payload() returns a 
# list with one item for each part. The easiest way is to walk the message 
# and get the payload on each part:
# https://stackoverflow.com/questions/1463074/how-can-i-get-an-email-messages-text-content-using-python

# NOTE that a Message object consists of headers and payloads.

#create empty list storing course subject, number, and time of most recent notification
most_recent_notification = []
for response_part in msgs[-1]:
    if type(response_part) is tuple:
        my_msg=email.message_from_bytes((response_part[1]))
        for part in my_msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=False)
                #get day and time of email
                time = my_msg['Date']
                #find the word "Course" in the body
                course_index = body.find("Course:") + 8
                #get the course subject from course index to space
                course_subject = body[course_index:].split()[0]
                #get the course number from course index to space
                course_number = body[course_index:].split()[1]
                crn = body[course_index:].split()[3]
                #convert time to datetime object
                time = datetime.strptime(time, '%a, %d %b %Y %H:%M:%S %z')
                most_recent_notification = (course_subject, course_number, crn, time)
print(most_recent_notification)


('KIN', '104', '35023', datetime.datetime(2022, 12, 30, 3, 5, 8, tzinfo=datetime.timezone.utc))


In [357]:
#check if the most recent notification is within the last 10 minutes
def check_event(most_recent_notification):
    utc=pytz.UTC
    most_recent_time = most_recent_notification[3]
    time_now = utc.localize(datetime.utcnow())
    return most_recent_time > time_now - timedelta(minutes=10)


In [358]:
check_event(most_recent_notification)

False

In [359]:
# create selenium driver
# https://banner.apps.uillinois.edu/StudentRegistrationSSB/ssb/registration?mepCode=1UIUC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait





In [360]:
# create driver
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://banner.apps.uillinois.edu/StudentRegistrationSSB/ssb/registration?mepCode=1UIUC")


Exception ignored in: <function Service.__del__ at 0x7f7fc89f0700>
Traceback (most recent call last):
  File "/Users/vishalrajpal/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/common/service.py", line 182, in __del__
    try:
KeyboardInterrupt: 
[WDM] - Downloading: 100%|██████████| 8.61M/8.61M [00:01<00:00, 8.67MB/s]


In [ ]:

#click on the "Register" button


In [ ]:
#click on the "Register" button
button = driver.find_element(By.ID, "registerLink")
button.click()

In [ ]:
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)
netID, netIDPassword = my_credentials["netID"], my_credentials["illinoisPassword"]
#login with netID and password
netID_input = driver.find_element(By.ID, "netid")
netID_input.send_keys(netID)
netIDPassword_input = driver.find_element(By.ID, "easpass")
netIDPassword_input.send_keys(netIDPassword)
netIDPassword_input.send_keys(Keys.RETURN)


In [ ]:
#click on select term
select_term = driver.find_element(By.ID, "select2-chosen-1")
select_term.click()
t.sleep(1)
#select term
term = driver.find_element(By.ID, "120231")
term.click()
#click on submit
submit = driver.find_element(By.ID, "term-go")
submit.click()

In [ ]:
#click on enter CRN
t.sleep(1)
enter_CRN = driver.find_element(By.ID, "enterCRNs-tab")
enter_CRN.click()


In [ ]:
course_crn = most_recent_notification[2]
#enter CRN
crn_input = driver.find_element(By.ID, "txt_crn1")
crn_input.send_keys(course_crn)
#click add to summary
add_to_summary = driver.find_element(By.ID, "addCRNbutton")
add_to_summary.click()
#wait 1 second for page to load
t.sleep(1)
submit = driver.find_element(By.ID, "saveButton")
submit.click()


In [ ]:
driver.close()
driver.quit()